# Lib

In [ ]:
from torchvision.datasets import CelebA
import os
import torch.nn as nn
import glob
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import math
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import math
import torchvision.transforms.functional as TF
import cv2
from keras.utils import image_dataset_from_directory
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.nn.parameter import Parameter

from torch.nn import init
from torchvision.utils import make_grid
from collections import OrderedDict

import torch
from torch.utils.data import Dataset

from skimage import transform
import skimage.io as io
import numpy as np
from torchvision.utils import save_image

from torchvision.datasets import MNIST

In [ ]:
os.chdir("/content/drive/MyDrive/AD Last Research")

# Helper Functions

In [ ]:
from sklearn.manifold import TSNE

def get_2d_tsne(x):
    tsne = TSNE(n_components=2, random_state=0,  learning_rate='auto', init='random')
    x_2d = tsne.fit_transform(x)

    return x_2d

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 128, 128)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
def ploter(image, image_hat):
    """
    (H, W)
    """
    plt.figure()
    plt.subplot(1,2,1)
    #plt.imshow(image_hat, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image_hat)
    plt.tight_layout()
    plt.title("Reconstruct")

    plt.subplot(1,2,2)
    #plt.imshow(image, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image)
    plt.tight_layout()
    plt.title("Original")

    plt.show()

# Gen

In [ ]:
from mymodels.generator import Generator_XL, Generator_S

In [ ]:
generator = Generator_S(features=64,
                        img_size=32,
                        latent_dim=100,
                        channels=3)
count_parameters(generator)

# Disc

In [ ]:
from mymodels.discriminator import Discriminator

In [ ]:
discriminator = Discriminator(features=32, channels=3, img_size=32, optimization='gan')

In [ ]:
count_parameters(discriminator)

# Data

In [ ]:
from dataset import customdataset

In [ ]:
img_size = 32
channels = 1
batch_size = 10

num_data = 1000

normal_label = 8
ano_label = 2

In [ ]:
# Train
train_dataset_normal = customdataset.get_nomral_dataset_train(dataset_name='mnist',
                                                               label=normal_label,
                                                               img_size=img_size, channels=channels,
                                                               data_path='./MNIST', download=True)

train_indx = SubsetRandomSampler(list(range(num_data)))
train_loader_normal = DataLoader(train_dataset_normal, batch_size=batch_size, sampler=train_indx)

In [ ]:
# Test
test_dataset_normal,  test_dataset_ano = customdataset.get_dataset_test(dataset_name='mnist',
                                                                         normal_label=normal_label, ano_label=ano_label,
                                                                         img_size=img_size, channels=channels,
                                                                         data_path="./MNIST", download=True)

test_loader_normal = DataLoader(test_dataset_normal, batch_size = test_dataset_normal.__len__())
test_loader_ano = DataLoader(test_dataset_ano, batch_size = test_dataset_ano.__len__())

# Vanilla GAN

### Hyperparameters

In [ ]:
device = 'cuda'
latent_dim = 100

### All in One

In [ ]:
# Nets
generator = Generator_S(features=64,
                        img_size=img_size,
                        latent_dim=latent_dim,
                        channels=channels).to(device)

discriminator = Discriminator(features=16, channels=channels, img_size=img_size, optimization='gan', last_bias = False).to(device)

In [ ]:
# Loss & Optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters())
optimizer_D = optim.Adam(discriminator.parameters())

### Re-Init

In [ ]:
# WRONG

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
generator = generator.apply(weights_init_normal)
discriminator = discriminator.apply(weights_init_normal)

### Train

In [ ]:
save_tsne = {}
total_G_loss = 0.0
total_D_loss = 0.0

In [ ]:
device = 'cuda'

init_epoch = 615
num_epochs = 1000
sample_interval = 25

In [ ]:
"""
info = torch.load("LAST_Vanilla_GAN.pt")
init_epoch = info['epoch']
generator.load_state_dict(info['state_dict_G'])
discriminator.load_state_dict(info['state_dict_D'])
optimizer_G.load_state_dict(info['optimizer_G'])
optimizer_D.load_state_dict(info['optimizer_D'])
save_tsne = info['save_tsne']
"""

In [ ]:
for _, (real_images, _) in enumerate(test_loader_normal):
  real_images_test = real_images.cpu()

In [ ]:
torch.cuda.empty_cache()
for epoch in range(init_epoch, num_epochs+1):
    torch.cuda.empty_cache()
    # Load Train Data ##############################################
    for i, (real_images, _) in enumerate(train_loader_normal):
        torch.cuda.empty_cache()
        # Preprocessing ##############################################
        valid = torch.FloatTensor(real_images.shape[0], 1).fill_(1.0).to(device)
        fake = torch.FloatTensor(real_images.shape[0], 1).fill_(0.0).to(device)
        real_images = real_images.to(device)
        generator = generator.to(device)
        discriminator = discriminator.to(device)

        # Train Generator ########################################
        optimizer_G.zero_grad()
        z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
        gen_imgs = generator(z)

        g_loss_adv = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss = g_loss_adv
        g_loss.backward()
        optimizer_G.step()
        total_G_loss += g_loss.cpu().detach().numpy()


        # Train Discriminator ########################################
        optimizer_D.zero_grad()
        discriminator_opinion_real = discriminator(real_images)
        discriminator_opinion_fake = discriminator(gen_imgs.detach())
        real_loss = adversarial_loss(discriminator_opinion_real, valid)
        fake_loss = adversarial_loss(discriminator_opinion_fake, fake)
        d_loss_adv = (real_loss + fake_loss) / 2
        d_loss = d_loss_adv

        d_loss.backward()
        optimizer_D.step()
        total_D_loss += d_loss.cpu().detach().numpy()


    # Load Test Data ##############################################
    torch.cuda.empty_cache()
    if epoch % sample_interval ==0:
      z = torch.FloatTensor(np.random.normal(0, 1, (real_images_test.shape[0], latent_dim))).cpu()
      generator = generator.cpu()
      gen_imgs = generator(z)

      # Visualization ##############################################
      show_tensor_images(gen_imgs, size=(channels, img_size, img_size))
      show_tensor_images(real_images_test, size=(channels, img_size, img_size))

      num = math.floor(np.random.uniform(0, len(test_loader_normal)))
      ploter(real_images_test.detach().cpu()[num][0], gen_imgs.detach().cpu()[num][0])

      # Save t-SNE ##############################################
      discriminator = discriminator.cpu()
      features_real = discriminator.conv_layers(real_images_test).flatten(1)
      features_fake = discriminator.conv_layers(gen_imgs).flatten(1)

      features_real_2D = get_2d_tsne(features_real.detach().cpu())
      features_fake_2D = get_2d_tsne(features_fake.detach().cpu())

      save_tsne[f'epoch_{epoch}'] = {'features_real_2D' : features_real_2D, 'features_fake_2D' : features_fake_2D}

      torch.save({"epoch": epoch,
      "state_dict_G": generator.state_dict(),
      "state_dict_D": discriminator.state_dict(),
      "optimizer_G": optimizer_G.state_dict(),
      "optimizer_D": optimizer_D.state_dict(),
      "save_tsne" : save_tsne
      }, "LAST_Vanilla_GAN.pt")

        # Losses ##############################################
        # SAVE ##############################################

In [ ]:
epoch

In [ ]:
for i in range(0, 615, 25):
  features_real_2D = save_tsne[f'epoch_{i}']['features_real_2D']
  features_fake_2d = save_tsne[f'epoch_{i}']['features_fake_2D']
  plt.figure()
  plt.scatter(features_real_2D[:, 0], features_real_2D[:, 1], label='Real')
  plt.scatter(features_fake_2D[:, 0], features_fake_2D[:, 1], label='Fake')
  plt.legend()
  plt.savefig(f"./tsne_s/epoch_{i}.png")

In [ ]:
"""
features_real = discriminator.conv_layers(real_images).flatten(1)
features_fake = discriminator.conv_layers(gen_imgs).flatten(1)

features_real_2D = get_2d_tsne(features_real.detach().cpu())
features_fake_2D = get_2d_tsne(features_fake.detach().cpu())
"""

plt.figure()
plt.scatter(features_real_2D[:, 0], features_real_2D[:, 1], label='Real')
plt.scatter(features_fake_2D[:, 0], features_fake_2D[:, 1], label='Fake')
plt.legend()
plt.savefig(f"./tsne_s_epoch_{i}.png")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')